### 🔹 Step 1: Setup & Imports

In [4]:
# Imports
import pandas as pd
import numpy as np
import os

# Define folder paths
base_path = r'C:\Users\moein\anaconda3\4 - Python Fundamentals\02 Data'
prepared_path = os.path.join(base_path, 'Prepared Data')

# Load the previously merged dataframe
file_path_2 = os.path.join(prepared_path, 'ords_prods_merge.pkl')
ords_prods_merge = pd.read_pickle(file_path_2)

# Check shape and structure
ords_prods_merge.shape
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,merge_flag,price_label,busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,0.0,196,1,0,both,Soda,77,7,9.0,both,Mid-range product,Regular days,Average orders
1,2539329,1,1,2,8,0.0,14084,2,0,both,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both,Mid-range product,Regular days,Average orders
2,2539329,1,1,2,8,0.0,12427,3,0,both,Original Beef Jerky,23,19,4.4,both,Low-range product,Regular days,Average orders
3,2539329,1,1,2,8,0.0,26088,4,0,both,Aged White Cheddar Popcorn,23,19,4.7,both,Low-range product,Regular days,Average orders
4,2539329,1,1,2,8,0.0,26405,5,0,both,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both,Low-range product,Regular days,Average orders


### 🔹 Step 2: Aggregated Mean of “order_number” by “department_id”

In [11]:
# Mean order_number grouped by department_id (entire dataset)
dept_mean_orders = ords_prods_merge.groupby('department_id')['order_number'].mean()
print(dept_mean_orders)

department_id
1     15.457687
2     17.277920
3     17.179756
4     17.811403
5     15.213779
6     16.439806
7     17.225773
8     15.340520
9     15.895474
10    20.197148
11    16.170828
12    15.887622
13    16.583304
14    16.757377
15    16.165037
16    17.663250
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64


The values presented above represent the mean order number per department for the entire dataset. Compared to the previous analysis using the 1-million-row subset, we can make the following observations:

- The mean values are generally higher in the full dataset. This suggests that when we include more data (i.e., more users and more orders), the average number of orders per department increases slightly.

- The rank order of departments remains mostly consistent, indicating that popular departments (like department 4 – likely "produce") still lead in number of orders.

- Departments like 21 and 10 still show significantly higher average order numbers, possibly due to specialized or recurring products (like personal care or baby products).

This illustrates why working with full data is often necessary to get a more representative picture and avoid under- or over-estimating certain metrics.


### 🔹 Step 3: Create Loyalty Flag Using transform() and loc()

In [6]:
# Create max_order column for each user
ords_prods_merge['max_order'] = ords_prods_merge.groupby('user_id')['order_number'].transform(np.max)

# Assign loyalty flag based on max_order
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

# Check frequency
ords_prods_merge['loyalty_flag'].value_counts()

C:\Users\moein\AppData\Local\Temp\ipykernel_16376\66720588.py:2: FutureWarning: The provided callable <function max at 0x0000025ED544C4A0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby('user_id')['order_number'].transform(np.max)


loyalty_flag
Regular customer    15891077
Loyal customer      10293737
New customer         6249398
Name: count, dtype: int64

### 🔹 Step 4: Analyze Spending by Loyalty Flag

In [7]:
# Basic statistics of prices by loyalty category
loyalty_price_stats = ords_prods_merge.groupby('loyalty_flag')['prices'].describe()
print(loyalty_price_stats)

                       count       mean         std  min  25%  50%   75%  \
loyalty_flag                                                               
Loyal customer    10293737.0  10.388747  327.864108  1.0  4.2  7.4  11.2   
New customer       6249398.0  13.294370  597.301692  1.0  4.2  7.4  11.3   
Regular customer  15891077.0  12.496203  539.478009  1.0  4.2  7.4  11.3   

                      max  
loyalty_flag               
Loyal customer    99999.0  
New customer      99999.0  
Regular customer  99999.0  


The descriptive statistics above compare the product prices purchased by different customer loyalty segments.

Key Insights:

- Loyal customers have the lowest average price per item at €10.39, compared to Regular customers (€12.50) and New customers (€13.29).

- The median price (50%) is identical across all segments (€7.4), indicating that while the typical purchase is similar, the higher averages for New and Regular customers may be due to occasional more expensive purchases (right-skewed distributions).

- ll groups share the same minimum (€1.0) and maximum (€99,999.0) values, which suggests extreme outliers exist in all segments. However, the standard deviation increases notably from Loyal → Regular → New, indicating more price variability among new customers.

✔️ Loyal customers tend to buy slightly cheaper items, possibly reflecting regular, habitual purchases of essentials.

✔️ New customers spend more on average, which could be due to bulk buying, exploratory purchases, or one-off high-value items.

These insights could help the marketing team tailor campaigns—for example, promote **higher-value products to loyal customers** or **offer discounts to new users to encourage continued use**.

### 🔹 Step 5: Create Spending Flag Based on Average Price per User

In [8]:
# Create mean price per user
ords_prods_merge['avg_price'] = ords_prods_merge.groupby('user_id')['prices'].transform('mean')

# Create spending_flag column
ords_prods_merge.loc[ords_prods_merge['avg_price'] < 10, 'spending_flag'] = 'Low spender'
ords_prods_merge.loc[ords_prods_merge['avg_price'] >= 10, 'spending_flag'] = 'High spender'

# Optional: Check results
ords_prods_merge['spending_flag'].value_counts()

spending_flag
Low spender     31798751
High spender      635461
Name: count, dtype: int64

### 🔹 Step 6: Create Order Frequency Flag

In [9]:
# Create median days since prior order per user
ords_prods_merge['median_days_since_prior'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform('median')

# Create order frequency flag
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior'] > 20, 'frequency_flag'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['median_days_since_prior'] > 10) & (ords_prods_merge['median_days_since_prior'] <= 20), 'frequency_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior'] <= 10, 'frequency_flag'] = 'Frequent customer'

# Check value counts
ords_prods_merge['frequency_flag'].value_counts()

frequency_flag
Frequent customer        22814871
Regular customer          6929354
Non-frequent customer     2689987
Name: count, dtype: int64

### 🔹 Step 7: Export Final Dataframe

In [10]:
# Export final dataframe with all new flags
output_path = r"C:\Users\moein\anaconda3\4 - Python Fundamentals\02 Data\Prepared Data\ords_prods_merge_enriched.pkl"
ords_prods_merge.to_pickle(output_path)